In [1]:
import geopandas as gpd
from tqdm import tqdm
import libpysal as lp
import json
import pandas as pd
from matplotlib.patches import Patch  # used for the plot legends later

###  Get the data in the folder ./LCPS_data for pre-processing. Let's call this `retrieved data`.

In [2]:
def retrieve_data(sy):
    """
    Retrieve data for a given school year (sy)
    """
    data_dir = "./LCPS_data"
    # Read the data files
    schools = gpd.read_file('{}/LCPS_Sites_{}.shp'.format(data_dir, sy))
    students = gpd.read_file('{}/Students_{}.shp'.format(data_dir, sy))
    spas = gpd.read_file('{}/PlanningZones_{}.shp'.format(data_dir, sy))
    
    return schools, students, spas


In [3]:
sy = '2017_2018'

In [4]:
# raw_schools, raw_students, raw_spas = retrieve_data(sy)

###  Get the processed data from the folder ./data

In [5]:
#spas_processed = gpd.read_file('new_SPAs_2017_2018.json', driver='GeoJSON')
spas = gpd.read_file('./data/new_SPAs_2017_2018.json', driver='GeoJSON')
students = gpd.read_file('./data/new_Students_2017_2018.json', driver='GeoJSON')
schools = gpd.read_file('./data/new_Schools_2017_2018.json', driver='GeoJSON')




#new line to run the newly downloaded json files, should move around anywhere.
#new line to run the newly downloaded json files, should move around anywhere.

In [6]:
#students.head()
#schools.head()
spas.head()

#uncomment the one you want to see

,OBJECTID,COUNT_,SPA,ELEM_,INT_,HIGH_,ELEM_CODE,MID_CODE,HIGH_CODE,DISTRICT,...,HIGH_POP_N,HIGH_POP_P,TOT_POP_A,TOT_POP_W,TOT_POP_M2,TOT_POP_B,TOT_POP_H,TOT_POP_N,TOT_POP_P,geometry
0,1,5.0,WL03.4,126,205,311,LOV,HRM,WHS,WL,...,0,0,1,37,0,2,7,0,0,"POLYGON ((11725868.086 7152149.054, 11725870.6..."
1,2,5.0,WL03,126,205,311,LOV,HRM,WHS,WL,...,0,0,0,83,5,0,6,1,0,"POLYGON ((11726805.147 7160565.439, 11726826.6..."
2,3,5.0,WL27.2,118,202,305,HAM,BRM,LVH,WL,...,0,0,0,11,0,1,5,0,0,"POLYGON ((11720159.464 7099502.139, 11720172.1..."
3,4,5.0,WL32.1,132,205,311,MTV,HRM,WHS,WL,...,0,0,8,205,14,8,23,1,0,"POLYGON ((11694466.485 7097608.657, 11694418.5..."
4,5,5.0,WL02,126,205,311,LOV,HRM,WHS,WL,...,0,0,9,170,3,0,5,0,0,"POLYGON ((11715899.154 7166497.655, 11715932.1..."


## Analysis Using New SPAs Data

In [7]:
spas.keys()

Index(['OBJECTID', 'COUNT_', 'SPA', 'ELEM_', 'INT_', 'HIGH_', 'ELEM_CODE',
       'MID_CODE', 'HIGH_CODE', 'DISTRICT', 'UTILITIES', 'SHAPE_Leng',
       'SHAPE_Area', 'STDYAREA_1', 'DISTRICT_1', 'PLANNING_Z', 'ELEM', 'MID',
       'HIGH', 'UTILITIE_1', 'DISTRICT_2', 'MID_2018', 'HIGH_2018', 'PK', 'KG',
       'GR1', 'GR2', 'GR3', 'GR4', 'GR5', 'GR6', 'GR7', 'GR8', 'GR9', 'GR10',
       'GR11', 'GR12', 'TOTAL_KG_5', 'TOTAL_6_8', 'TOTAL_9_12', 'TOTAL_KG_1',
       'TOTAL_PK_1', 'ELEM_POP', 'MID_POP', 'HIGH_POP', 'TOT_POP',
       'ELEM_POP_A', 'ELEM_POP_W', 'ELEM_POP_M2', 'ELEM_POP_B', 'ELEM_POP_H',
       'ELEM_POP_N', 'ELEM_POP_P', 'MID_POP_A', 'MID_POP_W', 'MID_POP_M2',
       'MID_POP_B', 'MID_POP_H', 'MID_POP_N', 'MID_POP_P', 'HIGH_POP_A',
       'HIGH_POP_W', 'HIGH_POP_M2', 'HIGH_POP_B', 'HIGH_POP_H', 'HIGH_POP_N',
       'HIGH_POP_P', 'TOT_POP_A', 'TOT_POP_W', 'TOT_POP_M2', 'TOT_POP_B',
       'TOT_POP_H', 'TOT_POP_N', 'TOT_POP_P', 'geometry'],
      dtype='object')

In [8]:
# Choose possible grade level; ELEM, MID, HIGH for SPAs

grade = 'MID'
asians = '{}_POP_A'.format(grade)
whites = '{}_POP_W'.format(grade)
mixed = '{}_POP_M2'.format(grade)
blacks = '{}_POP_B'.format(grade)
hispanics = '{}_POP_H'.format(grade)
natives = '{}_POP_N'.format(grade)
pacifics = '{}_POP_P'.format(grade)
non_whites = '{}_POP_NW'.format(grade)


spas_ethnic = spas[['{}_CODE'.format(grade) , asians, whites, mixed, blacks, hispanics, natives, pacifics, '{}_POP'.format(grade)]]
spas_ethnic.loc[:, non_whites] = spas_ethnic.loc[:, asians] +  spas_ethnic.loc[:,blacks] +  spas_ethnic.loc[:,mixed] \
                            + spas_ethnic.loc[:,hispanics] + spas_ethnic.loc[:,natives] + spas_ethnic.loc[:,pacifics]
spas_grouped_raw = spas_ethnic.groupby(['{}_CODE'.format(grade) ]).sum()
spas_grouped = spas_grouped_raw.div(spas_grouped_raw['{}_POP'.format(grade)], axis=0)
spas_grouped.drop([asians, mixed, blacks, hispanics, natives, pacifics, '{}_POP'.format(grade)], axis='columns', inplace=True)
spas_grouped.head()

#when generalizing one place to change and use the FORMAT style

C:\Users\james\Anaconda3\lib\site-packages\pandas\core\indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\james\Anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,MID_POP_W,MID_POP_NW
MID_CODE,,
BAM,0.480736,0.519264
BEM,0.552448,0.447552
BRM,0.789247,0.210753
ERM,0.513199,0.486801
FWS,0.466607,0.533393


In [9]:
#NOTE: Run this only once or start from the previous cell on subsequent runs - since we use iterrows and create a new column, the calculated value will change on the next runs

def get_measurements(df):
    from numpy import log
    import itertools
    
    # calculate absolute differences
    abs_diffs = []
    for row in df.iterrows():
    #     print([val for val in itertools.permutations(row[1], 2)])
    #     break
        diffs = [abs(val[1] - val[0]) for val in itertools.permutations(row[1], 2)]
        total = sum(diffs)/2
        abs_diffs.append(total)
    df.loc[:,'ABS_DIFF'] = abs_diffs

    # calculate dissimilarity index
    ind_diss = 0.5 * sum(abs(df[whites]/df[whites].sum() - df[non_whites]/df[non_whites].sum()))

    # calculate entropy index
    entropy = -(df[non_whites]*log(df[non_whites]) + df[whites]*log(df[whites]))
    entropy.replace(float('NaN'),0, inplace=True)

    df.loc[:,'ENTROPY_IND'] = entropy

    return df, ind_diss



In [10]:
measurements_df, ind_diss_sa1 = get_measurements(spas_grouped)
segregated_abs_diff = measurements_df[measurements_df['ABS_DIFF'] > 0.7]
segregated_entropy = measurements_df[measurements_df['ENTROPY_IND'] < 0.4]
print(ind_diss_sa1)
segregated_entropy

0.22744901692196107


,MID_POP_W,MID_POP_NW,ABS_DIFF,ENTROPY_IND
MID_CODE,,,,


In [11]:
# measurements_df.loc[interested_districts,:]
measurements_df

,MID_POP_W,MID_POP_NW,ABS_DIFF,ENTROPY_IND
MID_CODE,,,,
BAM,0.480736,0.519264,0.038529,0.692405
BEM,0.552448,0.447552,0.104895,0.687636
BRM,0.789247,0.210753,0.578495,0.514952
ERM,0.513199,0.486801,0.026398,0.692799
FWS,0.466607,0.533393,0.066785,0.690915
HPM,0.540863,0.459137,0.081726,0.689804
HRM,0.825472,0.174528,0.650943,0.462994
JLS,0.605461,0.394539,0.210923,0.670735
JML,0.469982,0.530018,0.060036,0.691344


In [12]:
ax, props = spas_grouped.boxplot(column=[whites, non_whites], figsize=(12,8), return_type='both')

## Conclusions for Original Data


Overall, using the dissimilarity index (~ 0.2), there does not appear to be any segregation within the districts.

Individually, using the absolute differences (0.7 cut-off) and entropy index (0.4 cut-off), there is no evidence of segregation in any district.


In [13]:
segregated_abs_diff

,MID_POP_W,MID_POP_NW,ABS_DIFF,ENTROPY_IND
MID_CODE,,,,


In [14]:
segregated_entropy

,MID_POP_W,MID_POP_NW,ABS_DIFF,ENTROPY_IND
MID_CODE,,,,


In [15]:
# legend_handles = [Patch(facecolor='red', label='GUI'),
#                 Patch(facecolor='green', label='LIN'),
#                 Patch(facecolor='yellow', label='SUG'),
#                 Patch(facecolor='orange', label='SUL')]

# ax = spas.boundary.plot(figsize=(15,14))
# spas[spas['ELEM_CODE'] == 'GUI'].plot(ax=ax, color='red')
# spas[spas['ELEM_CODE'] == 'LIN'].plot(ax=ax, color='green')
# spas[spas['ELEM_CODE'] == 'SUG'].plot(ax=ax, color='yellow')
# spas[spas['ELEM_CODE'] == 'SUL'].plot(ax=ax, color='orange')
# # ax.legend
# ax.legend(handles=legend_handles)

# Comparison to data generated by Algorithms

In [16]:
#spas_processed = gpd.read_file('new_SPAs_2017_2018.json', driver='GeoJSON')
# sa = json.loads('./results/SA/run7_ES_SA.json')
# shc = gpd.read_file('./results/SHC/run7_ES_SHC.json', driver='GeoJSON')
# ts = gpd.read_file('./results/TS/run7_ES_TS.json', driver='GeoJSON')

import json

def retrieve_output_data(algo, level, plan):
    output_df = pd.DataFrame({"SPA": [], "NEW_{}_CODE".format(grade): []})  #grade is a global variable
    result_dir = 'results'
    #Load the file as json. 
    with open('{}/{}/run7_{}_{}.json'.format(result_dir, algo, level, algo)) as file:
        #readlines() reads the file as string and loads() loads it into a dict
        obj = json.loads(''.join(file.readlines()))

        #Then iterate its items() as key value pairs
        for spa_code, elem_code in obj[str(plan)]['info']['Final']['zoneIds'].items():
            output_df.loc[len(output_df.index)] = [spa_code, elem_code]

    return output_df
#new line to run the newly downloaded json files, should move around anywhere.

In [17]:
import numpy as np
# interested_districts = ["GUI", "SUL", "LIN", "FHR", "HAM", "KWC", "WAT", "EVE", "FDE", "SUG"]
# data_array = np.zeros([25, 2*len(interested_districts)])

# Try to make dynamic later

In [18]:
def group_data(merged_spas):
    merged_spas_ethnic = merged_spas[['NEW_{}_CODE'.format(grade) , asians, whites, mixed, blacks, hispanics, natives, pacifics, '{}_POP'.format(grade), 'geometry']]
    merged_spas_ethnic.loc[:, non_whites] = merged_spas_ethnic.loc[:, asians] +  merged_spas_ethnic.loc[:,blacks] +  merged_spas_ethnic.loc[:,mixed] \
                                + merged_spas_ethnic.loc[:,hispanics] + merged_spas_ethnic.loc[:,natives] + merged_spas_ethnic.loc[:,pacifics]
    merged_spa_grouped_raw = merged_spas_ethnic.groupby(['NEW_{}_CODE'.format(grade) ]).sum()
    merged_spa_grouped = merged_spa_grouped_raw.div(merged_spa_grouped_raw['{}_POP'.format(grade)], axis=0)
    merged_spa_grouped.drop([asians, mixed, blacks, hispanics, natives, pacifics, '{}_POP'.format(grade)], axis='columns', inplace=True)
    return merged_spa_grouped


In [19]:
ind_diss = []

## Output Plan 1

### Simulated Annealing (SA)

In [20]:
plan = 1
algo = 'SA'
level = 'MS'
sa_output_df = retrieve_output_data(algo, level, plan)
new_spas_sa = pd.merge(spas, sa_output_df, on='SPA')
new_spas_sa_grouped = group_data(new_spas_sa)
# vals= np.concatenate([new_spas_sa_grouped.loc[i,:].values for i in interested_districts])
# data_array[plan-1,:] = vals

new_spas_sa_grouped.head()
#To change everything down to SA or TS

C:\Users\james\Anaconda3\lib\site-packages\pandas\core\indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\james\Anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,MID_POP_W,MID_POP_NW
NEW_MID_CODE,,
BAM,0.379671,0.620329
BEM,0.585758,0.414242
BRM,0.804651,0.195349
ERM,0.588235,0.411765
FWS,0.503982,0.496018


In [21]:
measurements_df, ind_diss_sa1 = get_measurements(new_spas_sa_grouped)
segregated_abs_diff_sa1 = measurements_df[measurements_df['ABS_DIFF'] > 0.7]
segregated_entropy_sa1 = measurements_df[measurements_df['ENTROPY_IND'] < 0.4]
ind_diss.append(ind_diss_sa1)
print(ind_diss_sa1)
segregated_entropy_sa1

0.27276650384127993


,MID_POP_W,MID_POP_NW,ABS_DIFF,ENTROPY_IND
NEW_MID_CODE,,,,


## Output Plan 2

### Simulated Annealing (SA)

In [22]:
plan = 2
sa_output_df = retrieve_output_data(algo, level, plan)
new_spas_sa = pd.merge(spas, sa_output_df, on='SPA')
new_spas_sa_grouped = group_data(new_spas_sa)
# vals= np.concatenate([new_spas_sa_grouped.loc[i,:].values for i in interested_districts])
# data_array[plan-1,:] = vals

new_spas_sa_grouped.head()
#To change everything down to SA or TS

,MID_POP_W,MID_POP_NW
NEW_MID_CODE,,
BAM,0.541505,0.458495
BEM,0.582651,0.417349
BRM,0.820881,0.179119
ERM,0.572960,0.427040
FWS,0.473247,0.526753


In [23]:
measurements_df, ind_diss_sa1 = get_measurements(new_spas_sa_grouped)
segregated_abs_diff_sa2 = measurements_df[measurements_df['ABS_DIFF'] > 0.7]
segregated_entropy_sa2 = measurements_df[measurements_df['ENTROPY_IND'] < 0.4]
ind_diss.append(ind_diss_sa1)
print(ind_diss_sa1)
segregated_entropy_sa2

0.2738353898277511


,MID_POP_W,MID_POP_NW,ABS_DIFF,ENTROPY_IND
NEW_MID_CODE,,,,
STM,0.136523,0.863477,0.726954,0.398601


## Output Plan 3

### Simulated Annealing (SA)

In [24]:
plan = 3
sa_output_df = retrieve_output_data(algo, level, plan)
new_spas_sa = pd.merge(spas, sa_output_df, on='SPA')
new_spas_sa_grouped = group_data(new_spas_sa)
# vals= np.concatenate([new_spas_sa_grouped.loc[i,:].values for i in interested_districts])
# data_array[plan-1,:] = vals

new_spas_sa_grouped.head()
#To change everything down to SA or TS

,MID_POP_W,MID_POP_NW
NEW_MID_CODE,,
BAM,0.416208,0.583792
BEM,0.579024,0.420976
BRM,0.807960,0.192040
ERM,0.530000,0.470000
FWS,0.458626,0.541374


In [25]:
measurements_df, ind_diss_sa1 = get_measurements(new_spas_sa_grouped)
segregated_abs_diff_sa3 = measurements_df[measurements_df['ABS_DIFF'] > 0.7]
segregated_entropy_sa3 = measurements_df[measurements_df['ENTROPY_IND'] < 0.4]
ind_diss.append(ind_diss_sa1)
print(ind_diss_sa1)
segregated_entropy_sa3

0.23929685636864656


,MID_POP_W,MID_POP_NW,ABS_DIFF,ENTROPY_IND
NEW_MID_CODE,,,,


## Output Plan 4

### Simulated Annealing (SA)

In [26]:
plan = 4
sa_output_df = retrieve_output_data(algo, level, plan)
new_spas_sa = pd.merge(spas, sa_output_df, on='SPA')
new_spas_sa_grouped = group_data(new_spas_sa)
# vals= np.concatenate([new_spas_sa_grouped.loc[i,:].values for i in interested_districts])
# data_array[plan-1,:] = vals

new_spas_sa_grouped.head()
#To change everything down to SA or TS

,MID_POP_W,MID_POP_NW
NEW_MID_CODE,,
BAM,0.444144,0.555856
BEM,0.571296,0.428704
BRM,0.816814,0.183186
ERM,0.556217,0.443783
FWS,0.478955,0.521045


In [27]:
measurements_df, ind_diss_sa1 = get_measurements(new_spas_sa_grouped)
segregated_abs_diff_sa4 = measurements_df[measurements_df['ABS_DIFF'] > 0.7]
segregated_entropy_sa4 = measurements_df[measurements_df['ENTROPY_IND'] < 0.4]
ind_diss.append(ind_diss_sa1)
print(ind_diss_sa1)
segregated_entropy_sa4

0.2582196160948335


,MID_POP_W,MID_POP_NW,ABS_DIFF,ENTROPY_IND
NEW_MID_CODE,,,,


## Output Plan 5

### Simulated Annealing (SA)

In [28]:
plan = 5
sa_output_df = retrieve_output_data(algo, level, plan)
new_spas_sa = pd.merge(spas, sa_output_df, on='SPA')
new_spas_sa_grouped = group_data(new_spas_sa)
# vals= np.concatenate([new_spas_sa_grouped.loc[i,:].values for i in interested_districts])
# data_array[plan-1,:] = vals

new_spas_sa_grouped.head()
#To change everything down to SA or TS

,MID_POP_W,MID_POP_NW
NEW_MID_CODE,,
BAM,0.355321,0.644679
BEM,0.534387,0.465613
BRM,0.811209,0.188791
ERM,0.542433,0.457567
FWS,0.496212,0.503788


In [29]:
measurements_df, ind_diss_sa1 = get_measurements(new_spas_sa_grouped)
segregated_abs_diff_sa5 = measurements_df[measurements_df['ABS_DIFF'] > 0.7]
segregated_entropy_sa5 = measurements_df[measurements_df['ENTROPY_IND'] < 0.4]
ind_diss.append(ind_diss_sa1)
print(ind_diss_sa1)
segregated_entropy_sa5

0.2644286545172282


,MID_POP_W,MID_POP_NW,ABS_DIFF,ENTROPY_IND
NEW_MID_CODE,,,,


## Output Plan 6

### Simulated Annealing (SA)

In [30]:
plan = 6
sa_output_df = retrieve_output_data(algo, level, plan)
new_spas_sa = pd.merge(spas, sa_output_df, on='SPA')
new_spas_sa_grouped = group_data(new_spas_sa)
# vals= np.concatenate([new_spas_sa_grouped.loc[i,:].values for i in interested_districts])
# data_array[plan-1,:] = vals

new_spas_sa_grouped.head()
#To change everything down to SA or TS

,MID_POP_W,MID_POP_NW
NEW_MID_CODE,,
BAM,0.556951,0.443049
BEM,0.523224,0.476776
BRM,0.821513,0.178487
ERM,0.495376,0.504624
FWS,0.522013,0.477987


In [31]:
measurements_df, ind_diss_sa1 = get_measurements(new_spas_sa_grouped)
segregated_abs_diff_sa6 = measurements_df[measurements_df['ABS_DIFF'] > 0.7]
segregated_entropy_sa6 = measurements_df[measurements_df['ENTROPY_IND'] < 0.4]
ind_diss.append(ind_diss_sa1)
print(ind_diss_sa1)
segregated_entropy_sa6

0.257360211708629


,MID_POP_W,MID_POP_NW,ABS_DIFF,ENTROPY_IND
NEW_MID_CODE,,,,


## Output Plan 7

### Simulated Annealing (SA)

In [32]:
plan = 7
sa_output_df = retrieve_output_data(algo, level, plan)
new_spas_sa = pd.merge(spas, sa_output_df, on='SPA')
new_spas_sa_grouped = group_data(new_spas_sa)
# vals= np.concatenate([new_spas_sa_grouped.loc[i,:].values for i in interested_districts])
# data_array[plan-1,:] = vals

new_spas_sa_grouped.head()
#To change everything down to SA or TS

,MID_POP_W,MID_POP_NW
NEW_MID_CODE,,
BAM,0.575641,0.424359
BEM,0.550872,0.449128
BRM,0.808442,0.191558
ERM,0.566879,0.433121
FWS,0.444845,0.555155


In [33]:
measurements_df, ind_diss_sa1 = get_measurements(new_spas_sa_grouped)
segregated_abs_diff_sa7 = measurements_df[measurements_df['ABS_DIFF'] > 0.7]
segregated_entropy_sa7 = measurements_df[measurements_df['ENTROPY_IND'] < 0.4]
ind_diss.append(ind_diss_sa1)
print(ind_diss_sa1)
segregated_entropy_sa7

0.2858867787481396


,MID_POP_W,MID_POP_NW,ABS_DIFF,ENTROPY_IND
NEW_MID_CODE,,,,


## Output Plan 8

### Simulated Annealing (SA)

In [34]:
plan = 8
sa_output_df = retrieve_output_data(algo, level, plan)
new_spas_sa = pd.merge(spas, sa_output_df, on='SPA')
new_spas_sa_grouped = group_data(new_spas_sa)
# vals= np.concatenate([new_spas_sa_grouped.loc[i,:].values for i in interested_districts])
# data_array[plan-1,:] = vals

new_spas_sa_grouped.head()
#To change everything down to SA or TS

,MID_POP_W,MID_POP_NW
NEW_MID_CODE,,
BAM,0.468405,0.531595
BEM,0.564539,0.435461
BRM,0.807122,0.192878
ERM,0.481261,0.518739
FWS,0.429630,0.570370


In [35]:
measurements_df, ind_diss_sa1 = get_measurements(new_spas_sa_grouped)
segregated_abs_diff_sa8 = measurements_df[measurements_df['ABS_DIFF'] > 0.7]
segregated_entropy_sa8 = measurements_df[measurements_df['ENTROPY_IND'] < 0.4]
ind_diss.append(ind_diss_sa1)
print(ind_diss_sa1)
segregated_entropy_sa8

0.2541572824239295


,MID_POP_W,MID_POP_NW,ABS_DIFF,ENTROPY_IND
NEW_MID_CODE,,,,


## Output Plan 9

### Simulated Annealing (SA)

In [36]:
plan = 9
sa_output_df = retrieve_output_data(algo, level, plan)
new_spas_sa = pd.merge(spas, sa_output_df, on='SPA')
new_spas_sa_grouped = group_data(new_spas_sa)
# vals= np.concatenate([new_spas_sa_grouped.loc[i,:].values for i in interested_districts])
# data_array[plan-1,:] = vals

new_spas_sa_grouped.head()
#To change everything down to SA or TS

,MID_POP_W,MID_POP_NW
NEW_MID_CODE,,
BAM,0.396512,0.603488
BEM,0.647677,0.352323
BRM,0.817374,0.182626
ERM,0.556399,0.443601
FWS,0.524786,0.475214


In [37]:
measurements_df, ind_diss_sa1 = get_measurements(new_spas_sa_grouped)
segregated_abs_diff_sa9 = measurements_df[measurements_df['ABS_DIFF'] > 0.7]
segregated_entropy_sa9 = measurements_df[measurements_df['ENTROPY_IND'] < 0.4]
ind_diss.append(ind_diss_sa1)
print(ind_diss_sa1)
segregated_entropy_sa9

0.2777403215271193


,MID_POP_W,MID_POP_NW,ABS_DIFF,ENTROPY_IND
NEW_MID_CODE,,,,


## Output Plan 10

### Simulated Annealing (SA)

In [38]:
plan = 10
sa_output_df = retrieve_output_data(algo, level, plan)
new_spas_sa = pd.merge(spas, sa_output_df, on='SPA')
new_spas_sa_grouped = group_data(new_spas_sa)
# vals= np.concatenate([new_spas_sa_grouped.loc[i,:].values for i in interested_districts])
# data_array[plan-1,:] = vals

new_spas_sa_grouped.head()
#To change everything down to SA or TS

,MID_POP_W,MID_POP_NW
NEW_MID_CODE,,
BAM,0.441687,0.558313
BEM,0.522537,0.477463
BRM,0.810834,0.189166
ERM,0.544368,0.455632
FWS,0.436559,0.563441


In [39]:
measurements_df, ind_diss_sa1 = get_measurements(new_spas_sa_grouped)
segregated_abs_diff_sa10 = measurements_df[measurements_df['ABS_DIFF'] > 0.7]
segregated_entropy_sa10 = measurements_df[measurements_df['ENTROPY_IND'] < 0.4]
ind_diss.append(ind_diss_sa1)
print(ind_diss_sa1)
segregated_entropy_sa10

0.2639503043620315


,MID_POP_W,MID_POP_NW,ABS_DIFF,ENTROPY_IND
NEW_MID_CODE,,,,


## Output Plan 11

### Simulated Annealing (SA)

In [40]:
plan = 11
sa_output_df = retrieve_output_data(algo, level, plan)
new_spas_sa = pd.merge(spas, sa_output_df, on='SPA')
new_spas_sa_grouped = group_data(new_spas_sa)
# vals= np.concatenate([new_spas_sa_grouped.loc[i,:].values for i in interested_districts])
# data_array[plan-1,:] = vals

new_spas_sa_grouped.head()
#To change everything down to SA or TS

,MID_POP_W,MID_POP_NW
NEW_MID_CODE,,
BAM,0.388095,0.611905
BEM,0.546218,0.453782
BRM,0.824546,0.175454
ERM,0.555006,0.444994
FWS,0.483204,0.516796


In [41]:
measurements_df, ind_diss_sa1 = get_measurements(new_spas_sa_grouped)
segregated_abs_diff_sa11 = measurements_df[measurements_df['ABS_DIFF'] > 0.7]
segregated_entropy_sa11 = measurements_df[measurements_df['ENTROPY_IND'] < 0.4]
ind_diss.append(ind_diss_sa1)
print(ind_diss_sa1)
segregated_entropy_sa11

0.2795335308226203


,MID_POP_W,MID_POP_NW,ABS_DIFF,ENTROPY_IND
NEW_MID_CODE,,,,


## Output Plan 12

### Simulated Annealing (SA)

In [42]:
plan = 12
sa_output_df = retrieve_output_data(algo, level, plan)
new_spas_sa = pd.merge(spas, sa_output_df, on='SPA')
new_spas_sa_grouped = group_data(new_spas_sa)
# vals= np.concatenate([new_spas_sa_grouped.loc[i,:].values for i in interested_districts])
# data_array[plan-1,:] = vals

new_spas_sa_grouped.head()
#To change everything down to SA or TS

,MID_POP_W,MID_POP_NW
NEW_MID_CODE,,
BAM,0.463215,0.536785
BEM,0.646825,0.353175
BRM,0.809641,0.190359
ERM,0.468543,0.531457
FWS,0.468391,0.531609


In [43]:
measurements_df, ind_diss_sa1 = get_measurements(new_spas_sa_grouped)
segregated_abs_diff_sa12 = measurements_df[measurements_df['ABS_DIFF'] > 0.7]
segregated_entropy_sa12 = measurements_df[measurements_df['ENTROPY_IND'] < 0.4]
ind_diss.append(ind_diss_sa1)
print(ind_diss_sa1)
segregated_entropy_sa12

0.25322657126803766


,MID_POP_W,MID_POP_NW,ABS_DIFF,ENTROPY_IND
NEW_MID_CODE,,,,


## Output Plan 13

### Simulated Annealing (SA)

In [44]:
plan = 13
sa_output_df = retrieve_output_data(algo, level, plan)
new_spas_sa = pd.merge(spas, sa_output_df, on='SPA')
new_spas_sa_grouped = group_data(new_spas_sa)
# vals= np.concatenate([new_spas_sa_grouped.loc[i,:].values for i in interested_districts])
# data_array[plan-1,:] = vals

new_spas_sa_grouped.head()
#To change everything down to SA or TS

,MID_POP_W,MID_POP_NW
NEW_MID_CODE,,
BAM,0.410513,0.589487
BEM,0.591391,0.408609
BRM,0.807292,0.192708
ERM,0.553555,0.446445
FWS,0.433766,0.566234


In [45]:
measurements_df, ind_diss_sa1 = get_measurements(new_spas_sa_grouped)
segregated_abs_diff_sa13 = measurements_df[measurements_df['ABS_DIFF'] > 0.7]
segregated_entropy_sa13 = measurements_df[measurements_df['ENTROPY_IND'] < 0.4]
ind_diss.append(ind_diss_sa1)
print(ind_diss_sa1)
segregated_entropy_sa13

0.2635066574597967


,MID_POP_W,MID_POP_NW,ABS_DIFF,ENTROPY_IND
NEW_MID_CODE,,,,


## Output Plan 14

### Simulated Annealing (SA)

In [46]:
plan = 14
sa_output_df = retrieve_output_data(algo, level, plan)
new_spas_sa = pd.merge(spas, sa_output_df, on='SPA')
new_spas_sa_grouped = group_data(new_spas_sa)
# vals= np.concatenate([new_spas_sa_grouped.loc[i,:].values for i in interested_districts])
# data_array[plan-1,:] = vals

new_spas_sa_grouped.head()
#To change everything down to SA or TS

,MID_POP_W,MID_POP_NW
NEW_MID_CODE,,
BAM,0.482902,0.517098
BEM,0.595458,0.404542
BRM,0.806132,0.193868
ERM,0.525802,0.474198
FWS,0.507246,0.492754


In [47]:
measurements_df, ind_diss_sa1 = get_measurements(new_spas_sa_grouped)
segregated_abs_diff_sa14 = measurements_df[measurements_df['ABS_DIFF'] > 0.7]
segregated_entropy_sa14 = measurements_df[measurements_df['ENTROPY_IND'] < 0.4]
ind_diss.append(ind_diss_sa1)
print(ind_diss_sa1)
segregated_entropy_sa14

0.251018419712678


,MID_POP_W,MID_POP_NW,ABS_DIFF,ENTROPY_IND
NEW_MID_CODE,,,,


## Output Plan 15

### Simulated Annealing (SA)

In [48]:
plan = 15
sa_output_df = retrieve_output_data(algo, level, plan)
new_spas_sa = pd.merge(spas, sa_output_df, on='SPA')
new_spas_sa_grouped = group_data(new_spas_sa)
# vals= np.concatenate([new_spas_sa_grouped.loc[i,:].values for i in interested_districts])
# data_array[plan-1,:] = vals

new_spas_sa_grouped.head()
#To change everything down to SA or TS

,MID_POP_W,MID_POP_NW
NEW_MID_CODE,,
BAM,0.422951,0.577049
BEM,0.560096,0.439904
BRM,0.807474,0.192526
ERM,0.395677,0.604323
FWS,0.533333,0.466667


In [49]:
measurements_df, ind_diss_sa1 = get_measurements(new_spas_sa_grouped)
segregated_abs_diff_sa15 = measurements_df[measurements_df['ABS_DIFF'] > 0.7]
segregated_entropy_sa15 = measurements_df[measurements_df['ENTROPY_IND'] < 0.4]
ind_diss.append(ind_diss_sa1)
print(ind_diss_sa1)
segregated_entropy_sa15

0.228959862836619


,MID_POP_W,MID_POP_NW,ABS_DIFF,ENTROPY_IND
NEW_MID_CODE,,,,


## Output Plan 16

### Simulated Annealing (SA)

In [50]:
plan = 16
sa_output_df = retrieve_output_data(algo, level, plan)
new_spas_sa = pd.merge(spas, sa_output_df, on='SPA')
new_spas_sa_grouped = group_data(new_spas_sa)
# vals= np.concatenate([new_spas_sa_grouped.loc[i,:].values for i in interested_districts])
# data_array[plan-1,:] = vals

new_spas_sa_grouped.head()
#To change everything down to SA or TS

,MID_POP_W,MID_POP_NW
NEW_MID_CODE,,
BAM,0.503244,0.496756
BEM,0.610473,0.389527
BRM,0.798482,0.201518
ERM,0.482522,0.517478
FWS,0.459665,0.540335


In [51]:
measurements_df, ind_diss_sa1 = get_measurements(new_spas_sa_grouped)
segregated_abs_diff_sa16 = measurements_df[measurements_df['ABS_DIFF'] > 0.7]
segregated_entropy_sa16 = measurements_df[measurements_df['ENTROPY_IND'] < 0.4]
ind_diss.append(ind_diss_sa1)
print(ind_diss_sa1)
segregated_entropy_sa16

0.2562538899870017


,MID_POP_W,MID_POP_NW,ABS_DIFF,ENTROPY_IND
NEW_MID_CODE,,,,


## Output Plan 17

### Simulated Annealing (SA)

In [52]:
plan = 17
sa_output_df = retrieve_output_data(algo, level, plan)
new_spas_sa = pd.merge(spas, sa_output_df, on='SPA')
new_spas_sa_grouped = group_data(new_spas_sa)
# vals= np.concatenate([new_spas_sa_grouped.loc[i,:].values for i in interested_districts])
# data_array[plan-1,:] = vals

new_spas_sa_grouped.head()
#To change everything down to SA or TS

,MID_POP_W,MID_POP_NW
NEW_MID_CODE,,
BAM,0.429395,0.570605
BEM,0.627563,0.372437
BRM,0.810280,0.189720
ERM,0.546069,0.453931
FWS,0.519465,0.480535


In [53]:
measurements_df, ind_diss_sa1 = get_measurements(new_spas_sa_grouped)
segregated_abs_diff_sa17 = measurements_df[measurements_df['ABS_DIFF'] > 0.7]
segregated_entropy_sa17 = measurements_df[measurements_df['ENTROPY_IND'] < 0.4]
ind_diss.append(ind_diss_sa1)
print(ind_diss_sa1)
segregated_entropy_sa17

0.23870634592895584


,MID_POP_W,MID_POP_NW,ABS_DIFF,ENTROPY_IND
NEW_MID_CODE,,,,


## Output Plan 18

### Simulated Annealing (SA)

In [54]:
plan = 18
sa_output_df = retrieve_output_data(algo, level, plan)
new_spas_sa = pd.merge(spas, sa_output_df, on='SPA')
new_spas_sa_grouped = group_data(new_spas_sa)
# vals= np.concatenate([new_spas_sa_grouped.loc[i,:].values for i in interested_districts])
# data_array[plan-1,:] = vals

new_spas_sa_grouped.head()
#To change everything down to SA or TS

,MID_POP_W,MID_POP_NW
NEW_MID_CODE,,
BAM,0.469407,0.530593
BEM,0.599483,0.400517
BRM,0.819594,0.180406
ERM,0.536386,0.463614
FWS,0.391453,0.608547


In [55]:
measurements_df, ind_diss_sa1 = get_measurements(new_spas_sa_grouped)
segregated_abs_diff_sa18 = measurements_df[measurements_df['ABS_DIFF'] > 0.7]
segregated_entropy_sa18 = measurements_df[measurements_df['ENTROPY_IND'] < 0.4]
ind_diss.append(ind_diss_sa1)
print(ind_diss_sa1)
segregated_entropy_sa18

0.27356817564103403


,MID_POP_W,MID_POP_NW,ABS_DIFF,ENTROPY_IND
NEW_MID_CODE,,,,


## Output Plan 19

### Simulated Annealing (SA)

In [56]:
plan = 19
sa_output_df = retrieve_output_data(algo, level, plan)
new_spas_sa = pd.merge(spas, sa_output_df, on='SPA')
new_spas_sa_grouped = group_data(new_spas_sa)
# vals= np.concatenate([new_spas_sa_grouped.loc[i,:].values for i in interested_districts])
# data_array[plan-1,:] = vals

new_spas_sa_grouped.head()
#To change everything down to SA or TS

,MID_POP_W,MID_POP_NW
NEW_MID_CODE,,
BAM,0.460400,0.539600
BEM,0.599005,0.400995
BRM,0.806481,0.193519
ERM,0.490442,0.509558
FWS,0.407487,0.592513


In [57]:
measurements_df, ind_diss_sa1 = get_measurements(new_spas_sa_grouped)
segregated_abs_diff_sa19 = measurements_df[measurements_df['ABS_DIFF'] > 0.7]
segregated_entropy_sa19 = measurements_df[measurements_df['ENTROPY_IND'] < 0.4]
ind_diss.append(ind_diss_sa1)
print(ind_diss_sa1)
segregated_entropy_sa19

0.25251354007183213


,MID_POP_W,MID_POP_NW,ABS_DIFF,ENTROPY_IND
NEW_MID_CODE,,,,


## Output Plan 20

### Simulated Annealing (SA)

In [58]:
plan = 20
sa_output_df = retrieve_output_data(algo, level, plan)
new_spas_sa = pd.merge(spas, sa_output_df, on='SPA')
new_spas_sa_grouped = group_data(new_spas_sa)
# vals= np.concatenate([new_spas_sa_grouped.loc[i,:].values for i in interested_districts])
# data_array[plan-1,:] = vals

new_spas_sa_grouped.head()
#To change everything down to SA or TS

,MID_POP_W,MID_POP_NW
NEW_MID_CODE,,
BAM,0.568338,0.431662
BEM,0.597206,0.402794
BRM,0.825794,0.174206
ERM,0.489566,0.510434
FWS,0.507740,0.492260


In [59]:
measurements_df, ind_diss_sa1 = get_measurements(new_spas_sa_grouped)
segregated_abs_diff_sa20 = measurements_df[measurements_df['ABS_DIFF'] > 0.7]
segregated_entropy_sa20 = measurements_df[measurements_df['ENTROPY_IND'] < 0.4]
ind_diss.append(ind_diss_sa1)
print(ind_diss_sa1)
segregated_entropy_sa20

0.22866395016836777


,MID_POP_W,MID_POP_NW,ABS_DIFF,ENTROPY_IND
NEW_MID_CODE,,,,


## Output Plan 21

### Simulated Annealing (SA)

In [60]:
plan = 21
sa_output_df = retrieve_output_data(algo, level, plan)
new_spas_sa = pd.merge(spas, sa_output_df, on='SPA')
new_spas_sa_grouped = group_data(new_spas_sa)
# vals= np.concatenate([new_spas_sa_grouped.loc[i,:].values for i in interested_districts])
# data_array[plan-1,:] = vals

new_spas_sa_grouped.head()
#To change everything down to SA or TS

,MID_POP_W,MID_POP_NW
NEW_MID_CODE,,
BAM,0.393939,0.606061
BEM,0.522161,0.477839
BRM,0.796444,0.203556
ERM,0.516981,0.483019
FWS,0.524712,0.475288


In [61]:
measurements_df, ind_diss_sa1 = get_measurements(new_spas_sa_grouped)
segregated_abs_diff_sa21 = measurements_df[measurements_df['ABS_DIFF'] > 0.7]
segregated_entropy_sa21 = measurements_df[measurements_df['ENTROPY_IND'] < 0.4]
ind_diss.append(ind_diss_sa1)
print(ind_diss_sa1)
segregated_entropy_sa21

0.22533402371504496


,MID_POP_W,MID_POP_NW,ABS_DIFF,ENTROPY_IND
NEW_MID_CODE,,,,


## Output Plan 22

### Simulated Annealing (SA)

In [62]:
plan = 22
sa_output_df = retrieve_output_data(algo, level, plan)
new_spas_sa = pd.merge(spas, sa_output_df, on='SPA')
new_spas_sa_grouped = group_data(new_spas_sa)
# vals= np.concatenate([new_spas_sa_grouped.loc[i,:].values for i in interested_districts])
# data_array[plan-1,:] = vals

new_spas_sa_grouped.head()
#To change everything down to SA or TS

,MID_POP_W,MID_POP_NW
NEW_MID_CODE,,
BAM,0.542918,0.457082
BEM,0.606089,0.393911
BRM,0.802579,0.197421
ERM,0.503261,0.496739
FWS,0.388268,0.611732


In [63]:
measurements_df, ind_diss_sa1 = get_measurements(new_spas_sa_grouped)
segregated_abs_diff_sa22 = measurements_df[measurements_df['ABS_DIFF'] > 0.7]
segregated_entropy_sa22 = measurements_df[measurements_df['ENTROPY_IND'] < 0.4]
ind_diss.append(ind_diss_sa1)
print(ind_diss_sa1)
segregated_entropy_sa22

0.2610493506067412


,MID_POP_W,MID_POP_NW,ABS_DIFF,ENTROPY_IND
NEW_MID_CODE,,,,


## Output Plan 23

### Simulated Annealing (SA)

In [64]:
plan = 23
sa_output_df = retrieve_output_data(algo, level, plan)
new_spas_sa = pd.merge(spas, sa_output_df, on='SPA')
new_spas_sa_grouped = group_data(new_spas_sa)
# vals= np.concatenate([new_spas_sa_grouped.loc[i,:].values for i in interested_districts])
# data_array[plan-1,:] = vals

new_spas_sa_grouped.head()
#To change everything down to SA or TS

,MID_POP_W,MID_POP_NW
NEW_MID_CODE,,
BAM,0.374199,0.625801
BEM,0.618314,0.381686
BRM,0.801164,0.198836
ERM,0.543850,0.456150
FWS,0.482963,0.517037


In [65]:
measurements_df, ind_diss_sa1 = get_measurements(new_spas_sa_grouped)
segregated_abs_diff_sa23 = measurements_df[measurements_df['ABS_DIFF'] > 0.7]
segregated_entropy_sa23 = measurements_df[measurements_df['ENTROPY_IND'] < 0.4]
ind_diss.append(ind_diss_sa1)
print(ind_diss_sa1)
segregated_entropy_sa23

0.2557985315663538


,MID_POP_W,MID_POP_NW,ABS_DIFF,ENTROPY_IND
NEW_MID_CODE,,,,


## Output Plan 24

### Simulated Annealing (SA)

In [66]:
plan = 24
sa_output_df = retrieve_output_data(algo, level, plan)
new_spas_sa = pd.merge(spas, sa_output_df, on='SPA')
new_spas_sa_grouped = group_data(new_spas_sa)
# vals= np.concatenate([new_spas_sa_grouped.loc[i,:].values for i in interested_districts])
# data_array[plan-1,:] = vals

new_spas_sa_grouped.head()
#To change everything down to SA or TS

,MID_POP_W,MID_POP_NW
NEW_MID_CODE,,
BAM,0.527378,0.472622
BEM,0.634213,0.365787
BRM,0.806599,0.193401
ERM,0.454259,0.545741
FWS,0.507493,0.492507


In [67]:
measurements_df, ind_diss_sa1 = get_measurements(new_spas_sa_grouped)
segregated_abs_diff_sa24 = measurements_df[measurements_df['ABS_DIFF'] > 0.7]
segregated_entropy_sa24 = measurements_df[measurements_df['ENTROPY_IND'] < 0.4]
ind_diss.append(ind_diss_sa1)
print(ind_diss_sa1)
segregated_entropy_sa24

0.2422511225792635


,MID_POP_W,MID_POP_NW,ABS_DIFF,ENTROPY_IND
NEW_MID_CODE,,,,


## Output Plan 25

### Simulated Annealing (SA)

In [68]:
plan = 25
sa_output_df = retrieve_output_data(algo, level, plan)
new_spas_sa = pd.merge(spas, sa_output_df, on='SPA')
new_spas_sa_grouped = group_data(new_spas_sa)
# vals= np.concatenate([new_spas_sa_grouped.loc[i,:].values for i in interested_districts])
# data_array[plan-1,:] = vals

new_spas_sa_grouped.head()
#To change everything down to SA or TS

,MID_POP_W,MID_POP_NW
NEW_MID_CODE,,
BAM,0.664439,0.335561
BEM,0.571042,0.428958
BRM,0.796964,0.203036
ERM,0.509988,0.490012
FWS,0.520599,0.479401


In [69]:
measurements_df, ind_diss_sa1 = get_measurements(new_spas_sa_grouped)
segregated_abs_diff_sa25 = measurements_df[measurements_df['ABS_DIFF'] > 0.7]
segregated_entropy_sa25 = measurements_df[measurements_df['ENTROPY_IND'] < 0.4]
ind_diss.append(ind_diss_sa1)
print(ind_diss_sa1)
segregated_entropy_sa25

0.25953402517000707


,MID_POP_W,MID_POP_NW,ABS_DIFF,ENTROPY_IND
NEW_MID_CODE,,,,


## Conclusions for New Data after Algorithms


Overall, using the dissimilarity index (~ 0.2), there does not appear to be any segregation within the districts.

Individually, using the absolute differences (0.7 cut-off) and entropy index (0.4 cut-off), there seems to be no evidence of segregation in any districts. 


In [70]:
print(segregated_abs_diff_sa1)
print(segregated_abs_diff_sa2)
print(segregated_abs_diff_sa3)
print(segregated_abs_diff_sa4)
print(segregated_abs_diff_sa5)
print(segregated_abs_diff_sa6)
print(segregated_abs_diff_sa7)
print(segregated_abs_diff_sa8)
print(segregated_abs_diff_sa9)
print(segregated_abs_diff_sa10)
print(segregated_abs_diff_sa11)
print(segregated_abs_diff_sa12)
print(segregated_abs_diff_sa13)
print(segregated_abs_diff_sa14)
print(segregated_abs_diff_sa15)
print(segregated_abs_diff_sa16)
print(segregated_abs_diff_sa17)
print(segregated_abs_diff_sa18)
print(segregated_abs_diff_sa19)
print(segregated_abs_diff_sa20)
print(segregated_abs_diff_sa21)
print(segregated_abs_diff_sa22)
print(segregated_abs_diff_sa23)
print(segregated_abs_diff_sa24)
print(segregated_abs_diff_sa25)

Empty DataFrame
Columns: [MID_POP_W, MID_POP_NW, ABS_DIFF, ENTROPY_IND]
Index: []
              MID_POP_W  MID_POP_NW  ABS_DIFF  ENTROPY_IND
NEW_MID_CODE                                              
STM            0.136523    0.863477  0.726954     0.398601
Empty DataFrame
Columns: [MID_POP_W, MID_POP_NW, ABS_DIFF, ENTROPY_IND]
Index: []
Empty DataFrame
Columns: [MID_POP_W, MID_POP_NW, ABS_DIFF, ENTROPY_IND]
Index: []
Empty DataFrame
Columns: [MID_POP_W, MID_POP_NW, ABS_DIFF, ENTROPY_IND]
Index: []
Empty DataFrame
Columns: [MID_POP_W, MID_POP_NW, ABS_DIFF, ENTROPY_IND]
Index: []
Empty DataFrame
Columns: [MID_POP_W, MID_POP_NW, ABS_DIFF, ENTROPY_IND]
Index: []
Empty DataFrame
Columns: [MID_POP_W, MID_POP_NW, ABS_DIFF, ENTROPY_IND]
Index: []
Empty DataFrame
Columns: [MID_POP_W, MID_POP_NW, ABS_DIFF, ENTROPY_IND]
Index: []
Empty DataFrame
Columns: [MID_POP_W, MID_POP_NW, ABS_DIFF, ENTROPY_IND]
Index: []
Empty DataFrame
Columns: [MID_POP_W, MID_POP_NW, ABS_DIFF, ENTROPY_IND]
Index: []
Emp

In [71]:
print(segregated_entropy_sa1)
print(segregated_entropy_sa2)
print(segregated_entropy_sa3)
print(segregated_entropy_sa4)
print(segregated_entropy_sa5)
print(segregated_entropy_sa6)
print(segregated_entropy_sa7)
print(segregated_entropy_sa8)
print(segregated_entropy_sa9)
print(segregated_entropy_sa10)
print(segregated_entropy_sa11)
print(segregated_entropy_sa12)
print(segregated_entropy_sa13)
print(segregated_entropy_sa14)
print(segregated_entropy_sa15)
print(segregated_entropy_sa16)
print(segregated_entropy_sa17)
print(segregated_entropy_sa18)
print(segregated_entropy_sa19)
print(segregated_entropy_sa20)
print(segregated_entropy_sa21)
print(segregated_entropy_sa22)
print(segregated_entropy_sa23)
print(segregated_entropy_sa24)
print(segregated_entropy_sa25)

Empty DataFrame
Columns: [MID_POP_W, MID_POP_NW, ABS_DIFF, ENTROPY_IND]
Index: []
              MID_POP_W  MID_POP_NW  ABS_DIFF  ENTROPY_IND
NEW_MID_CODE                                              
STM            0.136523    0.863477  0.726954     0.398601
Empty DataFrame
Columns: [MID_POP_W, MID_POP_NW, ABS_DIFF, ENTROPY_IND]
Index: []
Empty DataFrame
Columns: [MID_POP_W, MID_POP_NW, ABS_DIFF, ENTROPY_IND]
Index: []
Empty DataFrame
Columns: [MID_POP_W, MID_POP_NW, ABS_DIFF, ENTROPY_IND]
Index: []
Empty DataFrame
Columns: [MID_POP_W, MID_POP_NW, ABS_DIFF, ENTROPY_IND]
Index: []
Empty DataFrame
Columns: [MID_POP_W, MID_POP_NW, ABS_DIFF, ENTROPY_IND]
Index: []
Empty DataFrame
Columns: [MID_POP_W, MID_POP_NW, ABS_DIFF, ENTROPY_IND]
Index: []
Empty DataFrame
Columns: [MID_POP_W, MID_POP_NW, ABS_DIFF, ENTROPY_IND]
Index: []
Empty DataFrame
Columns: [MID_POP_W, MID_POP_NW, ABS_DIFF, ENTROPY_IND]
Index: []
Empty DataFrame
Columns: [MID_POP_W, MID_POP_NW, ABS_DIFF, ENTROPY_IND]
Index: []
Emp

In [72]:
# legend_handles = [Patch(facecolor='black', label='HAM'),
#                   Patch(facecolor='green', label='LIN'),
#                   Patch(facecolor='orange', label='SUL'),
#                    Patch(facecolor='red', label='GUI'),
#                    Patch(facecolor='blue', label='WAT'),
#                    Patch(facecolor='magenta', label='FDE'),
#                    Patch(facecolor='cyan', label='FHR'),
#                    Patch(facecolor='yellow', label='EVE'),
#                    Patch(facecolor='tab:olive', label='KWC')
#                 ]

# ax = spas.boundary.plot(figsize=(15,14))
# spas[spas['ELEM_CODE'] == 'HAM'].plot(ax=ax, color='black')
# spas[spas['ELEM_CODE'] == 'LIN'].plot(ax=ax, color='green')
# spas[spas['ELEM_CODE'] == 'SUL'].plot(ax=ax, color='orange')

# spas[spas['ELEM_CODE'] == 'GUI'].plot(ax=ax, color='red')
# spas[spas['ELEM_CODE'] == 'WAT'].plot(ax=ax, color='blue')
# spas[spas['ELEM_CODE'] == 'FDE'].plot(ax=ax, color='magenta')

# spas[spas['ELEM_CODE'] == 'FHR'].plot(ax=ax, color='cyan')
# spas[spas['ELEM_CODE'] == 'EVE'].plot(ax=ax, color='yellow')
# spas[spas['ELEM_CODE'] == 'KWC'].plot(ax=ax, color='tab:olive')

# # ax.legend
# ax.legend(handles=legend_handles)

In [73]:
# from numpy import concatenate
# import pandas as pd

# col_level_1 = np.concatenate([[i]*2 for i in interested_districts])
# col_level_2 = ['W', 'NW'] * len(interested_districts)
# cols_combined = zip(col_level_1, col_level_2)
# col_index = pd.MultiIndex.from_tuples(cols_combined, names=["District", "Group"])
# row_index = pd.Index(range(1,26))
# combined_df = pd.DataFrame(data=data_array, index = row_index, columns = col_index)
# combined_df 

In [74]:
# data_raw = spas_grouped.loc[interested_districts, ['ELEM_POP_W','ELEM_POP_NW']]
# data_original = data_raw.values.reshape((1,len(interested_districts)*2))
# original_df =  pd.DataFrame(data=data_original, columns = col_index)
# # print(data_raw.values)
# original_df

In [75]:
# white_column=[('GUI','W'), ('SUL','W'), ('LIN','W'),  ('FHR','W'),  ('HAM','W'),  ('KWC','W'), ('WAT','W'), ('EVE','W'), ('FDE', 'W')]
# ax, props = combined_df.boxplot(column=white_column, figsize=(15,14), return_type='both', patch_artist=True)
# ax.set_title('White Percentages')
# ax.set_xlabel('Districts')
# ax.set_ylabel('Percentages')

In [76]:
# nonwhite_column=[('GUI','NW'), ('SUL','NW'), ('LIN','NW'),  ('FHR','NW'),  ('HAM','NW'),  ('KWC','NW'), ('WAT','NW'), ('EVE','NW'), ('FDE','NW')]
# ax, props = combined_df.boxplot(column=nonwhite_column, figsize=(15,14), return_type='both', patch_artist=True)
# ax.set_title('Non-White Percentages')
# ax.set_xlabel('Districts')
# ax.set_ylabel('Percentages')
# for box in props['boxes']:
#     box.set_facecolor('orange')

In [77]:
# import matplotlib.pyplot as plt

# ax, props = combined_df.boxplot(figsize=(15,14), return_type='both', patch_artist=True, manage_ticks=False)
# ax.set_title('Whites vs Non-whites Percentages')
# ax.set_xlabel('Districts')
# ax.set_ylabel('Percentages')
# num_boxes = len(props['boxes'])
# for i in range(1, num_boxes, 2):
#     props['boxes'][i].set_facecolor('orange')
    
# legend_handles = [Patch(facecolor='blue', label='White Pop'),
#                 Patch(facecolor='orange', label='Non-White Pop')]
# ax.legend(handles=legend_handles)

# plt.tick_params(labelbottom=False)

In [78]:
# ax = combined_df['GUI'].plot(marker='o', subplots=True)
# pic_folder = 'Combined_linegraphs_andboxplots'

# ax = combined_df['GUI'].plot(marker='o')
# ax.set_title("GUI")
# x = np.arange(1,26)
# yw = [spas_grouped.loc['GUI', 'ELEM_POP_W']] * 25
# ynw = [spas_grouped.loc['GUI', 'ELEM_POP_NW']] * 25
# ax.plot(x,yw,color='k', linewidth='3')
# ax.plot(x,ynw,color='c', linewidth='3')
# # ax.legend()
# ax.get_figure().savefig('GUI_compare_{}'.format(algo))

In [79]:
# ax =  combined_df['GUI'].boxplot(figsize=(8,6), patch_artist=True)
# ax.set_title('GUI')
# ax.set_xlabel('Groups')
# ax.set_ylabel('Percentages')


# ax,bp_key = original_df['GUI'].boxplot(ax=ax, return_type='both', patch_artist=True)
# for box in bp_key['boxes']:
#     box.set_color('red')
#     box.set_linewidth(3)
    
# ax.get_figure().savefig('GUI_combined_{}'.format(algo))

In [80]:
# ax = combined_df['SUL'].plot(marker='o')
# ax.set_title("SUL")
# yw = [spas_grouped.loc['SUL', 'ELEM_POP_W']] * 25
# ynw = [spas_grouped.loc['SUL', 'ELEM_POP_NW']] * 25
# ax.plot(x,yw,color='k', linewidth='3')
# ax.plot(x,ynw,color='c', linewidth='3')
# ax.get_figure().savefig('SUL_compare_{}'.format(algo))

In [81]:
# ax =  combined_df['SUL'].boxplot(figsize=(8,6), patch_artist=True)
# ax.set_title('SUL')
# ax.set_xlabel('Groups')
# ax.set_ylabel('Percentages')

# ax,bp_key = original_df['SUL'].boxplot(ax=ax, return_type='both', patch_artist=True)
# for box in bp_key['boxes']:
#     box.set_color('red')
#     box.set_linewidth(3)
    
# ax.get_figure().savefig('SUL_combined_{}'.format(algo))

In [82]:
# ax = combined_df['LIN'].plot(marker='o')
# ax.set_title("LIN")
# yw = [spas_grouped.loc['LIN', 'ELEM_POP_W']] * 25
# ynw = [spas_grouped.loc['LIN', 'ELEM_POP_NW']] * 25
# ax.plot(x,yw,color='k', linewidth='3')
# ax.plot(x,ynw,color='c', linewidth='3')
# ax.get_figure().savefig('LIN_compare_{}'.format(algo))

In [83]:
# ax =  combined_df['LIN'].boxplot(figsize=(8,6), patch_artist=True)
# ax.set_title('LIN')
# ax.set_xlabel('Groups')
# ax.set_ylabel('Percentages')

# ax,bp_key = original_df['LIN'].boxplot(ax=ax, return_type='both', patch_artist=True)
# for box in bp_key['boxes']:
#     box.set_color('red')
#     box.set_linewidth(3)
    
    
# ax.get_figure().savefig('LIN_combined_{}'.format(algo))

In [84]:
# ax = combined_df['FHR'].plot(marker='o')
# ax.set_title("FHR")
# yw = [spas_grouped.loc['FHR', 'ELEM_POP_W']] * 25
# ynw = [spas_grouped.loc['FHR', 'ELEM_POP_NW']] * 25
# ax.plot(x,yw,color='k', linewidth='3')
# ax.plot(x,ynw,color='c', linewidth='3')
# ax.get_figure().savefig('FHR_compare_{}'.format(algo))

In [85]:
# ax =  combined_df['FHR'].boxplot(figsize=(8,6), patch_artist=True)
# ax.set_title('FHR')
# ax.set_xlabel('Groups')
# ax.set_ylabel('Percentages')

# ax,bp_key = original_df['FHR'].boxplot(ax=ax, return_type='both', patch_artist=True)
# for box in bp_key['boxes']:
#     box.set_color('red')
#     box.set_linewidth(3)
    
    
# ax.get_figure().savefig('FHR_combined_{}'.format(algo))

In [86]:
# ax = combined_df['HAM'].plot(marker='o')
# ax.set_title("HAM")
# yw = [spas_grouped.loc['HAM', 'ELEM_POP_W']] * 25
# ynw = [spas_grouped.loc['HAM', 'ELEM_POP_NW']] * 25
# ax.plot(x,yw,color='k', linewidth='3')
# ax.plot(x,ynw,color='c', linewidth='3')

# ax.get_figure().savefig('HAM_compare_{}'.format(algo))

In [87]:
# ax =  combined_df['HAM'].boxplot(figsize=(8,6), patch_artist=True)
# ax.set_title('HAM')
# ax.set_xlabel('Groups')
# ax.set_ylabel('Percentages')

# ax,bp_key = original_df['HAM'].boxplot(ax=ax, return_type='both', patch_artist=True)
# for box in bp_key['boxes']:
#     box.set_color('red')
#     box.set_linewidth(3)
    
# ax.get_figure().savefig('HAM_combined_{}'.format(algo))

In [88]:
# ax = combined_df['KWC'].plot(marker='o')
# ax.set_title("KWC")
# yw = [spas_grouped.loc['KWC', 'ELEM_POP_W']] * 25
# ynw = [spas_grouped.loc['KWC', 'ELEM_POP_NW']] * 25
# ax.plot(x,yw,color='k', linewidth='3')
# ax.plot(x,ynw,color='c', linewidth='3')
# ax.get_figure().savefig('KWC_compare_{}'.format(algo))

In [89]:
# ax =  combined_df['KWC'].boxplot(figsize=(8,6), patch_artist=True)
# ax.set_title('KWC')
# ax.set_xlabel('Groups')
# ax.set_ylabel('Percentages')

# ax,bp_key = original_df['KWC'].boxplot(ax=ax, return_type='both', patch_artist=True)
# for box in bp_key['boxes']:
#     box.set_color('red')
#     box.set_linewidth(3)
    
# ax.get_figure().savefig('KWC_combined_{}'.format(algo))

In [90]:
# ax = combined_df['WAT'].plot(marker='o')
# ax.set_title("WAT")
# yw = [spas_grouped.loc['WAT', 'ELEM_POP_W']] * 25
# ynw = [spas_grouped.loc['WAT', 'ELEM_POP_NW']] * 25
# ax.plot(x,yw,color='k', linewidth='3')
# ax.plot(x,ynw,color='c', linewidth='3')
# ax.get_figure().savefig('WAT_compare_{}'.format(algo))

In [91]:
# ax =  combined_df['WAT'].boxplot(figsize=(8,6), patch_artist=True)
# ax.set_title('WAT')
# ax.set_xlabel('Groups')
# ax.set_ylabel('Percentages')

# ax,bp_key = original_df['WAT'].boxplot(ax=ax, return_type='both', patch_artist=True)
# for box in bp_key['boxes']:
#     box.set_color('red')
#     box.set_linewidth(3)
    
# ax.get_figure().savefig('WAT_combined_{}'.format(algo))

In [92]:
# ax = combined_df['EVE'].plot(marker='o')
# ax.set_title("EVE")
# yw = [spas_grouped.loc['EVE', 'ELEM_POP_W']] * 25
# ynw = [spas_grouped.loc['EVE', 'ELEM_POP_NW']] * 25
# ax.plot(x,yw,color='k', linewidth='3')
# ax.plot(x,ynw,color='c', linewidth='3')

# ax.get_figure().savefig('EVE_compare_{}'.format(algo))

In [93]:
# ax =  combined_df['EVE'].boxplot(figsize=(8,6), patch_artist=True)
# ax.set_title('EVE')
# ax.set_xlabel('Groups')
# ax.set_ylabel('Percentages')

# ax,bp_key = original_df['EVE'].boxplot(ax=ax, return_type='both', patch_artist=True)
# for box in bp_key['boxes']:
#     box.set_color('red')
#     box.set_linewidth(3)
    
# ax.get_figure().savefig('EVE_combined_{}'.format(algo))

In [94]:
# ax = combined_df['FDE'].plot(marker='o')
# ax.set_title("FDE")
# yw = [spas_grouped.loc['FDE', 'ELEM_POP_W']] * 25
# ynw = [spas_grouped.loc['FDE', 'ELEM_POP_NW']] * 25
# ax.plot(x,yw,color='k', linewidth='3')
# ax.plot(x,ynw,color='c', linewidth='3')

# ax.get_figure().savefig('FDE_compare_{}'.format(algo))

# this is for the original data, i think
# I can see that the original data seems to be optimized here, since the gap is closer than the output plans
# The original is generally good though, in most cases - just for this district and SA

In [96]:
# ax =  combined_df['FDE'].boxplot(figsize=(8,6), patch_artist=True)
# ax.set_title('FDE')
# ax.set_xlabel('Groups')
# ax.set_ylabel('Percentages')

# ax,bp_key = original_df['FDE'].boxplot(ax=ax, return_type='both', patch_artist=True)
# for box in bp_key['boxes']:
#     box.set_color('red')
#     box.set_linewidth(3)
    
# ax.get_figure().savefig('FDE_combined_{}'.format(algo))

In [ ]:
# ax = combined_df['SUG'].plot(marker='o')
# ax.set_title("SUG")
# yw = [spas_grouped.loc['SUG', 'ELEM_POP_W']] * 25
# ynw = [spas_grouped.loc['SUG', 'ELEM_POP_NW']] * 25
# ax.plot(x,yw,color='k', linewidth='3')
# ax.plot(x,ynw,color='c', linewidth='3')
# ax.get_figure().savefig('SUG_compare_{}'.format(algo))

In [97]:
# ax =  combined_df['SUG'].boxplot(figsize=(8,6), patch_artist=True)
# ax.set_title('SUG')
# ax.set_xlabel('Groups')
# ax.set_ylabel('Percentages')

# ax,bp_key = original_df['SUG'].boxplot(ax=ax, return_type='both', patch_artist=True)
# for box in bp_key['boxes']:
#     box.set_color('red')
#     box.set_linewidth(3)
    
# ax.get_figure().savefig('SUG_combined_{}'.format(algo))

In [ ]:
# combined_df['FDE'].hist()

In [ ]:
segregated_entropy
#this is for the original data

In [ ]:
ind_diss


In [ ]:
plt.hist(ind_diss, bins=[y/100 for y in range(28,40)])
# plt.axis([0, 25, 0.25, 0.50])
# plt.show()

#challenge with plotting histogram. Any help will be much appreciated